In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, hstack
import itertools
import xgboost as xgb
from xgboost import XGBClassifier
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LeakyReLU

In [ ]:
train_seq_df = pd.read_csv("train_text_seq.csv")
train_seq_X = train_seq_df['input_str'].tolist()
train_seq_Y = np.array(train_seq_df['label'].tolist())

val_seq_df = pd.read_csv("valid_text_seq.csv")
val_seq_X = val_seq_df['input_str'].tolist()
val_seq_Y = np.array(val_seq_df['label'].tolist())

In [ ]:
train_seq_X_list = [[int(digit) for digit in seq] for seq in train_seq_X]
val_seq_X_list = [ [int(digit) for digit in seq] for seq in val_seq_X]

In [ ]:
def one_hot_encode_sequence_flat(X):
    # Initialize an empty list to store the one-hot encoded vectors
    one_hot_encoded_flat = []

    for sequence in X:
        one_hot = np.zeros((len(sequence), 10))  # Create a 50x10 matrix for each string
        for i, digit in enumerate(sequence):
            one_hot[i, digit] = 1  # Set the corresponding digit index to 1
        one_hot_flat = one_hot.flatten()
        one_hot_encoded_flat.append(one_hot_flat)
    return np.array(one_hot_encoded_flat)

In [ ]:
train_seq_X_encoded = one_hot_encode_sequence_flat(train_seq_X_list)
val_seq_X_encoded = one_hot_encode_sequence_flat(val_seq_X_list)

In [ ]:
def create_model():
    # Input layer
    input_shape = (50,10)
    inputs = Input(shape=input_shape)
    x = Conv1D(32, kernel_size=3,activation='relu', kernel_regularizer=l2(0.01))(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=4)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)

    # Fully connected layers
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(x)

    x = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(x)

    # Output layer
    outputs = Dense(1, activation='sigmoid')(x)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
learning_rate = 0.001
batch_size = 64
epochs = 100

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)


In [ ]:
model = create_model()
model.fit(train_seq_X_encoded.reshape(-1, 50, 10), train_seq_Y, validation_data=(val_seq_X_encoded.reshape(-1, 50, 10), val_seq_Y), epochs = epochs, batch_size=batch_size, callbacks=[reduce_lr])


Epoch 1/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5054 - loss: 1.5950 - val_accuracy: 0.5297 - val_loss: 1.2173 - learning_rate: 0.0010
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5889 - loss: 1.1258 - val_accuracy: 0.6196 - val_loss: 0.9737 - learning_rate: 0.0010
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7009 - loss: 0.8496 - val_accuracy: 0.6769 - val_loss: 0.8320 - learning_rate: 0.0010
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7488 - loss: 0.7008 - val_accuracy: 0.7239 - val_loss: 0.7228 - learning_rate: 0.0010
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7873 - loss: 0.5968 - val_accuracy: 0.7648 - val_loss: 0.6308 - learning_rate: 0.0010
Epoch 6/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8123 - loss: 0.5325 - val_accuracy: 0.8282 - val_loss: 0.5375 - learning_rate: 0.0010
Epoch 7/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8317 - loss: 0

In [ ]:
model.save("trained_model_for_dataset_3.keras")

In [ ]:
model3 = tf.keras.models.load_model('trained_model_for_dataset_3.keras')
y_pred = model3.predict(val_seq_X_encoded.reshape(-1, 50, 10))
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy_val = accuracy_score(val_seq_Y, y_pred_binary)
accuracy_val

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


0.885480572597137